In [17]:
import pandas as pd
import numpy as np
import clipboard, re

title = ['발파진동(cm/s)', '진동레벨dB(V)', '소음레벨dB(A)']

In [18]:
# parse process
# 1. ExtractColumns ( remove Columns in Clipboard items )
# 2. ConversionErrorValue ( Conversion Value of Error to np.NAN )
# 3. ClassficationByDate ( Classfication by Date Value )
# 4. DeleteOtherValue ( Delete Useless Value )
# 5. ExtractLocation ( Extract Location Value to List )
# 6. ListToDict ( Conversion List to Dictionary )

In [19]:
# 0. Copy List ( This process is implemented for test )
clipboard_list = list(filter(lambda v: v, clipboard.paste().replace("\r\n", " ").split(" ")))

In [20]:
# 1. ExtractColumns
columns = []

for i in clipboard_list:
    if isinstance(i, str) and not bool(re.search(r'\d', i)):
        columns.append(i)
    else:
        break

    items_list = list(filter(lambda x: x not in columns, clipboard_list))

print(columns)

['일시', '발파횟수', '시간', '장약량(kg)', '발파위치', '발파진동및소음측정치(max값)', '지발당장약량', '총장약량', 'STA', '발파진동(cm/s)', '진동레벨dB(V)', '소음레벨dB(A)', '측정위치']


In [21]:
# 2. ConversionErrorValue
for x, i in enumerate(items_list):
    if i == "계측기오류":
        for j in range(0, 3):
            if j < 1:
                items_list.pop(x + j)
            items_list.insert(x + j, "계측기 오류")

print(items_list)

['1월2일', '1회', '10:02', '1.6-2.0', '210.0', '7k+642.25~644.75(종점)', '0.014', '48.83', '70.17', '수양버들식당', '2회', '16:18', '0.4-1.2', '95.0', '5k+244.15~242.90(시점)', 'N/T', 'N/T', 'N/T', '안산현대2차아파트상가', '0.1', '64.28', '59.98', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '3회', '18:47', '1.6-2.0', '205.0', '7k+644.75~647.25(종점)', '0.052', '59.02', '53.69', '수양버들식당', '계', '510.0', '1월3일', '1회', '7:01', '0.4-1.2', '110.0', '5k+242.90~241.65(시점)', '0.051', '54.12', '55.34', '안산현대2차아파트상가', 'N/T', 'N/T', 'N/T', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '2회', '10:34', '1.6-2.0', '245.0', '7k+644.75~647.25(종점)', '0.016', '48.65', '71.06', '수양버들식당', '3회', '18:18', '0.4-1.2', '95.0', '5k+241.65~240.40(시점)', '0.014', '48.42', '51.46', '안산현대2차아파트상가', '0.09', '63.55', '64.31', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '계', '450.0', '1월4일', '1회', '7:01', '1.6-2.0', '220.0', '7k+647.25~649.75(종점)', 'N/T', 'N/T', 'N/T', '안산천주교회', '2회', '9:43', '0.4-1.2', '100.0', '5k+239.15~

In [22]:
# 3. ClassficationByDate
section = []
current_section = []

for i in items_list:
    if re.match(r'\d+월\d+일', i):
        if current_section:
            section.append(current_section)
            current_section = []
    current_section.append(i)

if current_section:
    section.append(current_section)

print(section)

[['1월2일', '1회', '10:02', '1.6-2.0', '210.0', '7k+642.25~644.75(종점)', '0.014', '48.83', '70.17', '수양버들식당', '2회', '16:18', '0.4-1.2', '95.0', '5k+244.15~242.90(시점)', 'N/T', 'N/T', 'N/T', '안산현대2차아파트상가', '0.1', '64.28', '59.98', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '3회', '18:47', '1.6-2.0', '205.0', '7k+644.75~647.25(종점)', '0.052', '59.02', '53.69', '수양버들식당', '계', '510.0'], ['1월3일', '1회', '7:01', '0.4-1.2', '110.0', '5k+242.90~241.65(시점)', '0.051', '54.12', '55.34', '안산현대2차아파트상가', 'N/T', 'N/T', 'N/T', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '2회', '10:34', '1.6-2.0', '245.0', '7k+644.75~647.25(종점)', '0.016', '48.65', '71.06', '수양버들식당', '3회', '18:18', '0.4-1.2', '95.0', '5k+241.65~240.40(시점)', '0.014', '48.42', '51.46', '안산현대2차아파트상가', '0.09', '63.55', '64.31', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '계', '450.0'], ['1월4일', '1회', '7:01', '1.6-2.0', '220.0', '7k+647.25~649.75(종점)', 'N/T', 'N/T', 'N/T', '안산천주교회', '2회', '9:43', '0.4-1.2', '100.0', '5k+23

In [23]:
# 4. DelOtherValue
section2 = []

for j in section:
    temp_list = []
    first_key = None
    for x, i in enumerate(j):
        if first_key is None and re.match(r'\d.*?\.\d+[~-]\d+\.\d+', i):
            first_key = i
        elif first_key is not None and re.match(r'\d.*?\.\d+[~-]\d+\.\d+', i):
            first_key = None
        elif first_key is None and i != '계' and j[x - 1] != '계':
            temp_list.append(i)
    section2.append(temp_list)

print(section2)

[['1월2일', '1회', '10:02', '0.014', '48.83', '70.17', '수양버들식당', '2회', '16:18', 'N/T', 'N/T', 'N/T', '안산현대2차아파트상가', '0.1', '64.28', '59.98', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '3회', '18:47', '0.052', '59.02', '53.69', '수양버들식당'], ['1월3일', '1회', '7:01', '0.051', '54.12', '55.34', '안산현대2차아파트상가', 'N/T', 'N/T', 'N/T', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '2회', '10:34', '0.016', '48.65', '71.06', '수양버들식당', '3회', '18:18', '0.014', '48.42', '51.46', '안산현대2차아파트상가', '0.09', '63.55', '64.31', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동'], ['1월4일', '1회', '7:01', 'N/T', 'N/T', 'N/T', '안산천주교회', '2회', '9:43', '0.112', '60.83', '59.05', '안산현대2차아파트상가', '0.099', '63.46', '65.08', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트322동', '3회', '18:09', '0.076', '59.63', '62.94', '안산천주교회'], ['1월5일', '1회', '7:01', 'N/T', 'N/T', 'N/T', '안산천주교회', '2회', '9:56', '0.16', '66.52', '59.74', '주공1단지아파트110동', '0.101', '65.72', '65.85', '주공1단지아파트109동', 'N/T', 'N/T', 'N/T', '주공3단지아파트32

In [24]:
# 5. ExtractLocation
location = []
date_count, count_count, time_count = None, None, None
pass_count = 0

for i in section2:
    for j in i:
        # if j.replace("(", "").replace(")", "").isalnum() and not j.isdigit() and not re.match(r'\d+월\d+일', j) and not re.match(r'\d+회', j):
        if re.match(r'\d+월\d+일', j):
            date_count = True
            print(f'date_count : {j}')
        elif re.match(r'\d+회', j):
            count_count = True
            print(f'count_count : {j}')
        elif re.match(r'\d+:\d+', j):
            time_count = True
            print(f'time_count : {j}')
        elif date_count and count_count and time_count and pass_count == 3:
            location.append(j)
            pass_count = 0
        elif date_count and count_count and time_count:
            pass_count += 1
            print(f'pass_count : {j}')


location = list(set(location))

location

date_count : 1월2일
count_count : 1회
time_count : 10:02
pass_count : 0.014
pass_count : 48.83
pass_count : 70.17
count_count : 2회
time_count : 16:18
pass_count : N/T
pass_count : N/T
pass_count : N/T
pass_count : 0.1
pass_count : 64.28
pass_count : 59.98
pass_count : N/T
pass_count : N/T
pass_count : N/T
count_count : 3회
time_count : 18:47
pass_count : 0.052
pass_count : 59.02
pass_count : 53.69
date_count : 1월3일
count_count : 1회
time_count : 7:01
pass_count : 0.051
pass_count : 54.12
pass_count : 55.34
pass_count : N/T
pass_count : N/T
pass_count : N/T
pass_count : N/T
pass_count : N/T
pass_count : N/T
count_count : 2회
time_count : 10:34
pass_count : 0.016
pass_count : 48.65
pass_count : 71.06
count_count : 3회
time_count : 18:18
pass_count : 0.014
pass_count : 48.42
pass_count : 51.46
pass_count : 0.09
pass_count : 63.55
pass_count : 64.31
pass_count : N/T
pass_count : N/T
pass_count : N/T
date_count : 1월4일
count_count : 1회
time_count : 7:01
pass_count : N/T
pass_count : N/T
pass_count 

['주공1단지110동(현장)',
 '수양버들식당',
 '주공3단지아파트322동',
 '안산천주교회',
 '주공1단지아파트110동',
 '주공1단지아파트109동',
 '로얄프라자',
 '풍전빌딩',
 '안산현대2차아파트상가']

In [25]:
# 6. ListToDict
result = {}
value = []

date_key, time_key, location_key = None, None, None

for item in section2:
    for x, i in enumerate(item):
        if re.match(r'\d+월\d+일', i):
            date_key = i
        elif re.match(r'\d+회', i):
            count_key = i
        elif re.match(r'\d+:\d+', i):
            time_key = i
        elif i not in location:
            value.append(i)
        elif i in location:
            location_key = i
            result[location_key] = {} if location_key not in result else result[location_key]
            result[location_key][f'{time_key} | {count_key}'] = {'일시' : f'{date_key} {time_key}'}

            for j in range(len(title)):
                try:
                    tmp = float(value[j])
                except ValueError:
                    tmp = np.nan
                finally:
                    result[location_key][f'{time_key} | {count_key}'][title[j]] = tmp
            value = []

print(result)

            

{'수양버들식당': {'10:02 | 1회': {'일시': '1월2일 10:02', '발파진동(cm/s)': 0.014, '진동레벨dB(V)': 48.83, '소음레벨dB(A)': 70.17}, '18:47 | 3회': {'일시': '1월2일 18:47', '발파진동(cm/s)': 0.052, '진동레벨dB(V)': 59.02, '소음레벨dB(A)': 53.69}, '10:34 | 2회': {'일시': '1월3일 10:34', '발파진동(cm/s)': 0.016, '진동레벨dB(V)': 48.65, '소음레벨dB(A)': 71.06}}, '안산현대2차아파트상가': {'16:18 | 2회': {'일시': '1월2일 16:18', '발파진동(cm/s)': nan, '진동레벨dB(V)': nan, '소음레벨dB(A)': nan}, '7:01 | 1회': {'일시': '1월3일 7:01', '발파진동(cm/s)': 0.051, '진동레벨dB(V)': 54.12, '소음레벨dB(A)': 55.34}, '18:18 | 3회': {'일시': '1월3일 18:18', '발파진동(cm/s)': 0.014, '진동레벨dB(V)': 48.42, '소음레벨dB(A)': 51.46}, '9:43 | 2회': {'일시': '1월4일 9:43', '발파진동(cm/s)': 0.112, '진동레벨dB(V)': 60.83, '소음레벨dB(A)': 59.05}}, '주공1단지아파트109동': {'16:18 | 2회': {'일시': '1월2일 16:18', '발파진동(cm/s)': 0.1, '진동레벨dB(V)': 64.28, '소음레벨dB(A)': 59.98}, '7:01 | 1회': {'일시': '1월3일 7:01', '발파진동(cm/s)': nan, '진동레벨dB(V)': nan, '소음레벨dB(A)': nan}, '18:18 | 3회': {'일시': '1월3일 18:18', '발파진동(cm/s)': 0.09, '진동레벨dB(V)': 63.55, '소음레벨dB(A)': 64.31}, '9:4

In [26]:
df_list = []

for i in result:
    df_list.append(pd.DataFrame(result[i]).transpose())

In [27]:
def classficationEveningData(df):
    new_columns = []

    for x, i in enumerate(list(df.index)):
        time = int(df.loc[i, '일시'].split(" ")[1].split(":")[0])
        if time >= 18:
            new_columns.append(df.loc[i, '소음레벨dB(A)'])
            df.loc[i, '소음레벨dB(A)'] = np.nan
        else:
            new_columns.append(np.nan)

    df['After 18:00'] = new_columns
    df.loc['MIN'] = df.min()
    df.loc['MAX'] = df.max()

    return df

In [28]:
for i in df_list:
    print(classficationEveningData(i))

                    일시 발파진동(cm/s) 진동레벨dB(V) 소음레벨dB(A)  After 18:00
10:02 | 1회  1월2일 10:02      0.014     48.83     70.17          NaN
18:47 | 3회  1월2일 18:47      0.052     59.02       NaN        53.69
10:34 | 2회  1월3일 10:34      0.016     48.65     71.06          NaN
MIN         1월2일 10:02      0.014     48.65     70.17        53.69
MAX         1월3일 10:34      0.052     59.02     71.06        53.69
                    일시 발파진동(cm/s) 진동레벨dB(V) 소음레벨dB(A)  After 18:00
16:18 | 2회  1월2일 16:18        NaN       NaN       NaN          NaN
7:01 | 1회    1월3일 7:01      0.051     54.12     55.34          NaN
18:18 | 3회  1월3일 18:18      0.014     48.42       NaN        51.46
9:43 | 2회    1월4일 9:43      0.112     60.83     59.05          NaN
MIN         1월2일 16:18      0.014     48.42     55.34        51.46
MAX          1월4일 9:43      0.112     60.83     59.05        51.46
                     일시 발파진동(cm/s) 진동레벨dB(V) 소음레벨dB(A)  After 18:00
16:18 | 2회   1월2일 16:18        0.1     64.28     59.98       